<img src="http://193.175.187.164/static/OEP_logo_2_no_text.svg"  alt="OpenEnergy Platform" height="100" width="100"  align="left" /><img src="http://www.znes-flensburg.de/sites/default/files/Logo_ZNES_breit_farbig_1.png
"  alt="ZNES Flensburg" height="100" width="500"  align="right"/> 
# OpenEnergy Platform


In [ ]:
__copyright__ = "ZNES"
__license__ = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__ = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__ = "wolfbunke"

from IPython.core.display import HTML


# Tutorial - How to work with the OpenEnergy Database (oedb)


This Tutorial gives you an quick overview of the OpenEnergy Platform and how you can work with the **REST-full-HTTP** API in Python3. The full API Documentaion can be found on [ReadtheDocs.io](http://oep-data-interface.readthedocs.io/en/latest/api/how_to.html#authenticate "OpenEnergyPlatform’s documentation").


This Tutorial is seperated into two Parts. The first Part gives you an understanding of the work flow as well as an overview how to start with basic functions of the OpenEnergy Database API. 
The second Part shows you an Example how you can easly analyse data via the OpenEnergy Platform Database.

## Part I

* Get started - Sign-in and get your own token
* Create your own table
* Insert data to your table
* Get your data back

## Part II

* Get Power Plant data of Germany 
* Plot all German Power plants
* Make an easy analyse of the data

## Links and further information


How to work with Python and Jupyter Notebooks https://github.com/Open-Power-System-Data/common/wiki
http://nbviewer.jupyter.org/github/ocefpaf/folium_notebooks/blob/master/test_png_mpld3_vega_popup.ipynb

#### Overview of Packages:
http://geopandas.org/install.html#installing-geopandas 

http://docs.python-requests.org/en/master/


### ToDOs

* Create Envir
* create requirement


## Part I
 
### Get started - Sign-in and get your own token

In order to get the right you need to sign-in on the OpenEnergy Platform.
Go and click on the login Button and create a new account. The registration 
https://wiki.openmod-initiative.org/wiki/Special:RequestAccount


#### 1.  Click on the login Button and Sign in

<img src="images/login.png"  alt="OpenEnergy Platform login" height="100" width="100" align="left"/> 

#### 2.  Copy your Token

<img src="images/token.png"  alt="OpenEnergy Platform Token" height="400" width="500" align="left" /> 


## Create your own table


* You installed all Python Packages? Good, let's create our first Database table.


In [2]:
import requests
import pandas as pd

your_token = 'XXX'
oep_url= 'http://oep.iks.cs.ovgu.de/'


In [5]:
# Create date table 
data = { "query": 
        { "columns": [ { "name":"id", "data_type": "serial", "is_nullable": "NO" },
                      { "name":"name", "data_type": "varchar", "character_maximum_length": "50" },
                      { "name":"geom", "data_type": "geometry(point)" } ],
         "constraints": [ { "constraint_type": "PRIMARY KEY", "constraint_parameter": "id" } ] 
        } }


requests.put(oep_url+'/api/v0/schema/model_draft/tables/example_api_table_wolf_test/',
             json=data, headers={'Authorization': 'Token %s'%your_token} )

<Response [500]>

### Insert Data into your table

* Now we insert an random power plant from Germany with name and Point Geometry
* 

In [7]:
data = {"query": {"name": "Kraftwerke as", "geom": "0101000020E610000046B6F3FDD4082B403524EEB1F4094A40"}}
result = requests.post(oep_url+'/api/v0/schema/model_draft/tables/example_api_table_wolf_test/rows/new', json=data, headers={'Authorization': 'Token %s'%your_token} )
result.status_code

201

### Get your data back


In [8]:
get_data = requests.get(oep_url+'/api/v0/schema/model_draft/tables/example_api_table_wolf_test/rows/')
get_data.status_code

200

In [9]:
plant_df = pd.DataFrame(get_data.json())
plant_df

,geom,id,name
0,010100000046B6F3FDD4082B403524EEB1F4094A40,1,Kraftwerke as
1,010100000046B6F3FDD4082B403524EEB1F4094A40,2,Kraftwerke as
2,010100000046B6F3FDD4082B403524EEB1F4094A40,3,Kraftwerke as
3,010100000046B6F3FDD4082B403524EEB1F4094A40,4,Kraftwerke as


<table style="background-color:#FF7F50; width:100%;  border: 0px solid white;">
<tr><td> <center><h1>ToDo: </h1></center>

<br>  Add Metadata over OEP WEB-Interface: http://193.175.187.164/dataedit/view/model_draft/example_api_table_wolf_test
<br>  Add Graphic 
<br>  Add epsg:4326 to lon lat for plotting
   
   
   
   </td>
</tr>
</table>



In [10]:
import geopandas as gpd
from shapely.geometry import Point
import shapely.wkt
from shapely import wkb
from geoalchemy2.shape import to_shape


In [11]:
# https://gis.stackexchange.com/questions/119752/reading-postgis-geometry-with-shapely

plant_df.geom

#for parcel in parcels:
#   parcel_shape = to_shape(parcel.geom)
    
    
#plant_df.geom = wkb.loads(plant_df.geom, hex=True)
    
#crs = {'init': 'epsg:4326'}
#geo_df = gpd.GeoDataFrame(plant_df, crs=crs, geometry=geometry)

#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


0    010100000046B6F3FDD4082B403524EEB1F4094A40
1    010100000046B6F3FDD4082B403524EEB1F4094A40
2    010100000046B6F3FDD4082B403524EEB1F4094A40
3    010100000046B6F3FDD4082B403524EEB1F4094A40
Name: geom, dtype: object

## Part II

* Get Power Plant data of Germany 
* Plot all German Power plants
* Make an easy analyse of the data


In [12]:
conv_powerplants = requests.get(oep_url+'/api/v0/schema/supply/tables/ego_conventional_powerplant/rows/')
conv_powerplants.status_code

200

### Create Pandas DataFrame

In [13]:
df_plants = pd.DataFrame(conv_powerplants.json())

Let's take a look into our data:

In [14]:
df_plants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903 entries, 0 to 902
Data columns (total 32 columns):
block                    662 non-null object
bnetza_id                903 non-null object
capacity                 879 non-null float64
capacity_uba             222 non-null float64
chp                      880 non-null object
chp_capacity_uba         111 non-null float64
city                     903 non-null object
comment                  903 non-null object
commissioned             901 non-null float64
commissioned_original    902 non-null object
company                  900 non-null object
eeg                      903 non-null object
efficiency_data          137 non-null float64
efficiency_estimate      687 non-null float64
fuel                     903 non-null object
geom                     903 non-null object
gid                      903 non-null int64
lat                      903 non-null float64
lon                      903 non-null float64
name                     869 non-

In [15]:
df_plants.columns

Index(['block', 'bnetza_id', 'capacity', 'capacity_uba', 'chp',
       'chp_capacity_uba', 'city', 'comment', 'commissioned',
       'commissioned_original', 'company', 'eeg', 'efficiency_data',
       'efficiency_estimate', 'fuel', 'geom', 'gid', 'lat', 'lon', 'name',
       'name_uba', 'network_node', 'network_operator', 'postcode', 'retrofit',
       'shutdown', 'state', 'status', 'street', 'technology', 'type',
       'voltage'],
      dtype='object')

### Make Geo-plots
#### Point Plot 

In [16]:
import folium
from folium import plugins
import matplotlib.pyplot as plt
%matplotlib inline 

In [17]:
# define Map Region
map = folium.Map(location=[51, 9], zoom_start=6)

In [18]:
# Use column lon / lat in order to plot map
for name, row in df_plants.iloc[:1000].iterrows():
    folium.Marker([row["lat"], row["lon"]], popup=row["type"] ).add_to(map)
#map.create_map('stops.html')
map

#### Heat plot for locations

In [19]:
stops_heatmap = folium.Map(location=[51, 9], zoom_start=6)
stops_heatmap.add_child(plugins.HeatMap([[row["lat"], row["lon"]] for name, row in df_plants.iloc[:1000].iterrows()]))
stops_heatmap.save("heatmap.html")
stops_heatmap

### Make some statistics